# Module 1: Meme Generation

- Note: Desired Output: Meme template, Meme text, and caption (optional). Caption is just the text generation that we'll do in simple text generation module.

### Sub-Modules

0. Prerequisite: Need to create a meme templates table which contains 5-6 features that will be our user inputs (loc, audience, industry, etc) and contains 1 keywords col for each template.
    - 0.1: Get Meme names from API.
    - 0.2: Create prompt-based AI call function to get decided features for every meme template (e.g. Location, Audience, Keywords, etc). 
    - 0.3: Create dataset for all meme templates and their respective features.

1. Get input: Free-form and structured (will be getting it from user frontend).

2. Prepare input properly: keyword extraction for free-form, no preprocessing needed for structured. 

3. Perform Template matching by matching each column separately for every meme template (need a simple algo for this). For every meme template in our dataset:
    - 3.1: Match structured inputs first, one by one. Simply return boolean TRUE/FALSE -> A utility function needed to match strings (take care of capitalization). Simply return 1/0 for each column.
    - 3.2: Match the extracted keywords with keywords col in dataset, return percentage of matched keywords (e.g. 2/5 input-converted-tokeywords match with dataset keywords). Need 1 utility func for this -> returns a numerical value between 0-1 which represents the percentage.
    - 3.3: A final scoring func which simply returns the weighted sum of products $(Col1*weight1 + c2*w2 + ...)$
    - 3.4: Store this to a score-table (this will be created for every user input query). Sort the table in descending order and return the top 5 templates based on score.

4. Generate the meme text based on the template by using chat gpt api. One utility func needed to recieve [template name, user input, static prompt (this will be pre-set every generation)], and returns __formatted__ meme text (ensure it returns appropriate format for that meme template, and only returns 1 example).

5. Create a func to call meme-creation api ([link](https://rapidapi.com/meme-generator-api-meme-generator-api-default/api/meme-generator)): pass template-name and meme-text, receive created jpeg image (ensure a consistent format- png, jpg, jpeg, etc). Temporarily store while displaying to user, and if approved, store to user's db entry.

- Note: Steps 4 and 5 will be repeated for all 5 meme templates that were chosen as the highest scoring templates.

6. Give option for caption creation. Create a utility func using chatgpt and generate caption as well, based on the user input and static prompt (will need another static prompt for this func).

---

### Sub-Module 0

**Step 1:** Getting meme template names from the API that we're using ([Meme-Generator-Rapid-API](https://rapidapi.com/meme-generator-api-meme-generator-api-default/api/meme-generator))

In [1]:
import requests

url = "https://ronreiter-meme-generator.p.rapidapi.com/images"

headers = {
	"x-rapidapi-key": "",
	"x-rapidapi-host": "ronreiter-meme-generator.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

template_names = response.json()

In [8]:
print(template_names)

['10-Guy', '1950s-Middle-Finger', '1990s-First-World-Problems', '1st-World-Canadian-Problems', '2nd-Term-Obama', 'Aaaaand-Its-Gone', 'Ace-Primo', 'Actual-Advice-Mallard', 'Adalia-Rose', 'Admiral-Ackbar-Relationship-Expert', 'Advice-Dog', 'Advice-Doge', 'Advice-God', 'Advice-Peeta', 'Advice-Tam', 'Advice-Yoda', 'Afraid-To-Ask-Andy', 'Afraid-To-Ask-Andy-Closeup', 'Aint-Nobody-Got-Time-For-That', 'Alan-Greenspan', 'Alarm-Clock', 'Albert-Cagestein', 'Albert-Einstein-1', 'Alien-Meeting-Suggestion', 'Alright-Gentlemen-We-Need-A-New-Idea', 'Always-Has-Been', 'Alyssa-Silent-Hill', 'Am-I-The-Only-One-Around-Here', 'American-Chopper-Argument', 'Ancient-Aliens', 'And-everybody-loses-their-minds', 'And-then-I-said-Obama', 'Angry-Asian', 'Angry-Baby', 'Angry-Birds-Pig', 'Angry-Bride', 'Angry-Chef-Gordon-Ramsay', 'Angry-Chicken-Boss', 'Angry-Dumbledore', 'Angry-Koala', 'Angry-Rant-Randy', 'Angry-Toddler', 'Annoying-Childhood-Friend', 'Annoying-Facebook-Girl', 'Anri-Stares', 'Anti-Joke-Chicken', 'Apa

In [6]:
template_names.remove('poster')

In [7]:
len(template_names)

999

In [9]:
import pandas as pd

memes_df = pd.DataFrame(template_names, columns=['Name'])

In [11]:
memes_df.head(10)

,Name
0,10-Guy
1,1950s-Middle-Finger
2,1990s-First-World-Problems
3,1st-World-Canadian-Problems
4,2nd-Term-Obama
5,Aaaaand-Its-Gone
6,Ace-Primo
7,Actual-Advice-Mallard
8,Adalia-Rose
9,Admiral-Ackbar-Relationship-Expert


__Utility Func 0.1:__ ChatGPT Call for Feature Generation for Meme Template Dataset:

In [18]:
import openai
import json

client = openai.OpenAI(api_key=key)

meme_feature_extraction_function = {
    "name": "generate_meme_features",
    "description": "Generate structured marketing features for a given meme template.",
    "parameters": {
        "type": "object",
        "properties": {
            "meme_template": {
                "type": "string",
                "description": "The name of the meme template to generate features for."
            }
        },
        "required": ["meme_template"],
        "additionalProperties": False
    }
}

def extract_meme_features(meme_template):
    system_prompt = """
    You are an AI assistant that generates structured marketing features for meme templates.
    Each meme template is used for marketing purposes, particularly in social media advertising and brand engagement.

    You must return a structured JSON response with the following properties:

    - **audience**: The type of audience that best relates to the meme (e.g., Millennials, Gen Z, Professionals).
    - **humor_type**: The style of humor used in the meme (e.g., Relatable, Sarcastic, Dark, Corporate-friendly).
    - **industry**: The industries where this meme would work well (e.g., Fashion, Tech, Food).
    - **product**: Example of a product or service that could be marketed using this meme.
    - **platform**: The best social media platforms for this meme (e.g., Instagram, Twitter, LinkedIn, TikTok).
    - **marketing_goal**: The main marketing objective (e.g., Brand Awareness, Sales Promotion, Customer Engagement).
    - **emotion_targeted**: The core emotion that this meme triggers (e.g., FOMO, Excitement, Regret, Humor).
    - **engagement_type**: The kind of interaction this meme encourages (e.g., "Tag a Friend", "Share", "Comment").
    - **brand_tone**: The general brand voice that aligns with this meme (e.g., Playful, Luxury, Professional).
    - **meme_complexity**: Whether the meme is simple (easy to understand at a glance) or complex (requires some thinking).
    - **keywords**: A list of 10 relevant keywords associated with this meme.

    Ensure the output strictly follows the specified JSON format.
    """

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Generate features for the meme template: {meme_template}"}
        ],
        tools=[{"type": "function", "function": meme_feature_extraction_function}],
        tool_choice="required",
        response_format={"type": "json_object"}
    )

    if not completion.choices[0].message.tool_calls:
        raise ValueError("Function was not called by the model. Debugging required.")

    tool_call = completion.choices[0].message.tool_calls[0]

    try:
        response_data = json.loads(tool_call.function.arguments)
    except json.JSONDecodeError:
        raise ValueError("Function response is not valid JSON. Check OpenAI response.")

    return response_data


In [19]:
meme_template_name = "surprised-pikachu"
meme_features = extract_meme_features(meme_template_name)

In [20]:
print(json.dumps(meme_features, indent=4))

{
    "meme_template": "surprised-pikachu"
}
